# Assembled Notebook — SK Agents (BPMN Happy Path)
_Generated 2025-11-07T18:35:47.890495Z_

In [ ]:
# %% [SETUP]
# Package install (upgrade SK; remove pydrive2 to avoid OpenSSL conflicts on Colab)
!pip install -U semantic-kernel
!pip -q uninstall -y pydrive2

In [ ]:
# %% [SETUP-ENV]
import os, getpass
os.environ.setdefault('AZURE_OPENAI_ENDPOINT', 'https://4th-openai-resource.openai.azure.com')
os.environ.setdefault('AZURE_OPENAI_DEPLOYMENT', 'gpt-35-turbo')
os.environ.setdefault('AZURE_OPENAI_API_VERSION', '2024-10-21')
if not os.getenv('AZURE_OPENAI_API_KEY'):
    os.environ['AZURE_OPENAI_API_KEY'] = getpass.getpass('Enter AZURE_OPENAI_API_KEY (hidden): ').strip()
print('Azure OpenAI env ready.')

In [ ]:
# %% [KERNEL]
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()

service = AzureChatCompletion(
    service_id='azure',
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT'),  # your AOAI deployment name
    endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),           # e.g. https://<resource>.openai.azure.com
    # api_version=os.getenv('AZURE_OPENAI_API_VERSION'),   # optional
)
kernel.add_service(service)
print('Kernel ready (Azure OpenAI)')

In [ ]:
# %% [TOOLS]
def tool_azure_openai_llm(**kwargs):
    """Azure OpenAI / LLM integration placeholder."""
    # TODO: integrate SK chat/stream here
    return 'stub:Azure OpenAI / LLM ' + str(kwargs)

def tool_ai_search_rag_index(**kwargs):
    """AI Search (RAG Index) integration placeholder."""
    # TODO: azure-search-documents client example
    return 'stub:AI Search (RAG Index) ' + str(kwargs)

def tool_key_vault_config(**kwargs):
    """Key Vault / Config integration placeholder."""
    # TODO: azure-identity + azure-keyvault-secrets
    return 'stub:Key Vault / Config ' + str(kwargs)

TOOLS = {
    'tool_azure_openai_llm': tool_azure_openai_llm,
    'tool_ai_search_rag_index': tool_ai_search_rag_index,
    'tool_key_vault_config': tool_key_vault_config,
}
print('Tools:', list(TOOLS.keys()))

In [ ]:

# %% [AGENTS]
NODES = {
    "INT":  "Intent Detection / Topic Routing",
    "REQ":  "Invoke /orchestrate endpoint",
    "PLN":  "Plan Step – determine intent, choose tools",
    "ACT":  "Act Step – call tools / retrieve data",
    "REF":  "Reflect Step – validate, cite, ensure policy",
    "ANS":  "Compose Grounded Answer",
    "AOAI": "Azure OpenAI / LLM",
    "SEARCH":"AI Search (RAG Index)",
    "KV":   "Key Vault / Config",
}
EDGES = [
    ("ACT", "SEARCH"),
    ("ACT", "AOAI"),
    ("ACT", "KV"),
    ("REF", "AOAI"),
    ("ANS", "AOAI"),
]

def _slug(label: str) -> str:
    import re
    return re.sub(r"[^a-z0-9]+", "_", label.lower()).strip("_")

TARGET_TO_TOOL = {
    "AOAI":  "tool_azure_openai_llm",
    "SEARCH":"tool_ai_search_rag_index",
    "KV":    "tool_key_vault_config",
}

from collections import defaultdict
skills_by_agent = defaultdict(list)
for a, b in EDGES:
    if b in TARGET_TO_TOOL:
        skills_by_agent[a].append(TARGET_TO_TOOL[b])

import json

AGENTS = {}
for nid, label in NODES.items():
    slug = _slug(label)
    skills = sorted(set(skills_by_agent.get(nid, [])))
    system_prompt = f"You are the {label} capability agent."

    cls_src = f'''
class Agent_{slug}:
    def __init__(self, kernel):
        self.kernel = kernel
        self.name = "{label}"
        self.system_message = {json.dumps(system_prompt)}
        self.skills = {json.dumps(skills)}

    async def run(self, user_text: str) -> str:
        try:
            # DEMO cell will patch to force service_id='azure'.
            result = await self.kernel.invoke_prompt(self.system_message + "\\n\\nUser: " + user_text)
            return str(result)
        except Exception as e:
            return f"[{label} stub] Adjust SK call. Error: {{e}}"

    def available_tools(self):
        return [t for t in self.skills if t in TOOLS]

    def call(self, tool_name: str, **kwargs):
        fn = TOOLS.get(tool_name)
        if not fn:
            raise ValueError(f"Tool not found: {{tool_name}}")
        return fn(**kwargs)
'''
    ns = {}
    exec(cls_src, globals(), ns)
    AGENTS[f"agent_{slug}"] = ns[f"Agent_{slug}"](kernel)

print("Agents:", sorted(AGENTS.keys()))


In [ ]:
# %% [WIRES]
WIRES = {AGENTS[k].name: {'tools': AGENTS[k].skills} for k in AGENTS}
import json
print(json.dumps(WIRES, indent=2))

In [ ]:

# %% [DEMO]
import os, types, asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Ensure the Azure service is registered
try:
    kernel.remove_service("azure")
except Exception:
    pass

kernel.add_service(AzureChatCompletion(
    service_id="azure",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
))

# Patch all agents to force service_id='azure'
async def _run_with_azure(self, user_text: str):
    prompt = (getattr(self, "system_message", "") or "") + "\n\nUser: " + str(user_text)
    result = await self.kernel.invoke_prompt(prompt, service_id="azure")
    return str(result)

for name, agent in AGENTS.items():
    agent.kernel = kernel
    agent.run = types.MethodType(_run_with_azure, agent)

print("Patched agents:", list(AGENTS.keys()))

# Smoke test
async def demo():
    # Pick ACT if present (has tools), else first
    cand = [a for a in AGENTS.values() if "Act Step" in getattr(a, "name", "")]
    agent = cand[0] if cand else next(iter(AGENTS.values()))
    print("Agent:", agent.name)
    tools = agent.available_tools()
    print("Tools:", tools)
    if tools:
        print("Tool demo:", tools[0], "->", agent.call(tools[0], example="value"))

    print("LLM demo:")
    out = await agent.run("In one sentence, describe your role in this pipeline.")
    print(out)

await demo()
